In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [2]:
## BRISE UPOZORENJA
import warnings
warnings.filterwarnings('ignore')

In [3]:
# nakon što stavimo datoteke na Git uvijek čitati u ovomo obliku! (štedi vrijeme)
import time
start = time.time()
df = pd.read_hdf('./data/raw/training_dataset.h5', 'df')
end = time.time()
print(end - start)

# nakon što napravimo promjene kako treba, exportat ću ih u novu datoteku pomoću naredbe
# df.to_hdf('training_dataset2.h5', key='df')

8.49665355682373


In [4]:
### NAPOMENA S FAQ-a
df['PRIJEVREMENI_RASKID'] = np.where(df['DATUM_ZATVARANJA']>=(df['PLANIRANI_DATUM_ZATVARANJA']- np.timedelta64(10, 'D')), 'N', df['PRIJEVREMENI_RASKID'])

In [5]:
### BRISANJE PARTIJA S VIŠE KLIJENATA
old_shape = df.shape
gb = df.groupby('OZNAKA_PARTIJE')['KLIJENT_ID'].nunique() # baci broj jedinstvenih klijenata za pojedinu partiju
# party_labels = df2['OZNAKA_PARTIJE'].unique().tolist() 
mask = gb[df['OZNAKA_PARTIJE']] == 1 # gb[df2['OZNAKA_PARTIJE']] vrati za svaku oznaku partije (svaki redak tablice) njenu vrijednost i usporedi je s 1
df = df[ mask.values ] # mask je neki objekt koji moram ovako koristiti, rezultat je skup gdje za svaku partiju ima jedan klijent
print('Postotak originalnog dataset-a:')
print(df.shape[0]/old_shape[0])

Postotak originalnog dataset-a:
0.9988667707530188


In [6]:
df.shape

(5187239, 17)

In [7]:
df.head()

,DATUM_IZVJESTAVANJA,KLIJENT_ID,OZNAKA_PARTIJE,DATUM_OTVARANJA,PLANIRANI_DATUM_ZATVARANJA,DATUM_ZATVARANJA,UGOVORENI_IZNOS,STANJE_NA_KRAJU_PRETH_KVARTALA,STANJE_NA_KRAJU_KVARTALA,VALUTA,VRSTA_KLIJENTA,PROIZVOD,VRSTA_PROIZVODA,VISINA_KAMATE,TIP_KAMATE,STAROST,PRIJEVREMENI_RASKID
0,2013-12-31,412140,7146814,2012-08-27,2018-04-30,NaT,629155.97,639373.81,641562.01,2,1420,TM0109,L,1.50,A,22,N
1,2013-12-31,772139,5521579,2010-06-11,2017-06-30,NaT,46000.00,28481.94,26887.19,1,1410,FL0801,A,9.70,B,38,N
2,2013-12-31,481985,7443823,2013-01-23,2018-02-28,NaT,44679.50,44908.40,45062.09,2,1550,FD0100,L,3.10,A,37,N
3,2013-12-31,139014,6511639,2011-09-16,2016-09-30,NaT,29959.15,19691.64,18285.89,2,1410,FL0801,A,7.95,B,49,N
4,2013-12-31,952566,6143123,2011-05-23,2014-05-31,NaT,100000.00,24797.93,15685.33,1,1410,FL0801,A,9.70,B,35,N


In [8]:
df.isnull().sum()

DATUM_IZVJESTAVANJA                     0
KLIJENT_ID                              0
OZNAKA_PARTIJE                          0
DATUM_OTVARANJA                         0
PLANIRANI_DATUM_ZATVARANJA         420978
DATUM_ZATVARANJA                  3973234
UGOVORENI_IZNOS                         0
STANJE_NA_KRAJU_PRETH_KVARTALA     420917
STANJE_NA_KRAJU_KVARTALA                0
VALUTA                                  0
VRSTA_KLIJENTA                          0
PROIZVOD                                0
VRSTA_PROIZVODA                         0
VISINA_KAMATE                       51684
TIP_KAMATE                              0
STAROST                                 0
PRIJEVREMENI_RASKID                     0
dtype: int64

In [9]:
# kasnije cemo to srediti pa sada to stavljamo na NaN
df['STAROST'] = np.where((df['STAROST']<0) | (df['STAROST']>120), np.nan, df['STAROST'])

In [10]:
df.sort_values(['KLIJENT_ID', 'OZNAKA_PARTIJE', 'DATUM_OTVARANJA', 'DATUM_IZVJESTAVANJA'], inplace = True)

In [11]:
grouped = df.groupby(['KLIJENT_ID', 'OZNAKA_PARTIJE', 'DATUM_OTVARANJA'])

def max_value(series):
    return series.max()

df['UGOVORENI_IZNOS'] = grouped['UGOVORENI_IZNOS'].transform(max_value)


In [12]:
## IMPUTACIJA NEDOSTAJUCIH VRIJEDNOSTI

grouped1 = df.groupby(['KLIJENT_ID', 'OZNAKA_PARTIJE'])
grouped2 = df.groupby(['KLIJENT_ID', 'OZNAKA_PARTIJE', 'DATUM_OTVARANJA'])

def impute_backwards(series):
    return series.fillna(method = 'bfill')

def impute_mean(series):
    return series.fillna(series.mean())

df['AVG_VISINA_KAMATE'] = grouped1['VISINA_KAMATE'].transform(impute_mean)
df['NOVI_DZ'] = grouped1['DATUM_ZATVARANJA'].transform('last')
df['NOVI_PDZ']  = grouped1['PLANIRANI_DATUM_ZATVARANJA'].transform('last')
df['PRVI_DO']  = grouped1['DATUM_OTVARANJA'].transform('first')

# popunjavanje stanja na kraju prethodnog kvartala 

# df['NOVO_STANJE_NKPH'] = np.where(df['STANJE_NA_KRAJU_KVARTALA'].isnull(), df['UGOVORENI_IZNOS'], df['STANJE_NA_KRAJU_KVARTALA'])


In [13]:
df.isnull().sum()

DATUM_IZVJESTAVANJA                     0
KLIJENT_ID                              0
OZNAKA_PARTIJE                          0
DATUM_OTVARANJA                         0
PLANIRANI_DATUM_ZATVARANJA         420978
DATUM_ZATVARANJA                  3973234
UGOVORENI_IZNOS                         0
STANJE_NA_KRAJU_PRETH_KVARTALA     420917
STANJE_NA_KRAJU_KVARTALA                0
VALUTA                                  0
VRSTA_KLIJENTA                          0
PROIZVOD                                0
VRSTA_PROIZVODA                         0
VISINA_KAMATE                       51684
TIP_KAMATE                              0
STAROST                              3834
PRIJEVREMENI_RASKID                     0
AVG_VISINA_KAMATE                    4593
NOVI_DZ                           1306549
NOVI_PDZ                            14836
PRVI_DO                                 0
dtype: int64

In [14]:
output = df.copy()

output['PLANIRANI_DATUM_ZATVARANJA'] = output['NOVI_PDZ']
output['DATUM_ZATVARANJA'] = output['NOVI_DZ']
output['DATUM_OTVARANJA'] = output['PRVI_DO']
output['VISINA_KAMATE'] = output['AVG_VISINA_KAMATE']

del_col_list = ['NOVI_PDZ', 'NOVI_DZ', 'PRVI_DO', 'AVG_VISINA_KAMATE']
output.drop(del_col_list, axis = 1, inplace = True)

In [15]:
output.shape

(5187239, 17)

In [16]:
output2 = output[~(output['DATUM_ZATVARANJA'].isnull())]

In [17]:
output2.shape

(3880690, 17)

In [18]:
output2.isnull().sum()

DATUM_IZVJESTAVANJA                    0
KLIJENT_ID                             0
OZNAKA_PARTIJE                         0
DATUM_OTVARANJA                        0
PLANIRANI_DATUM_ZATVARANJA          7735
DATUM_ZATVARANJA                       0
UGOVORENI_IZNOS                        0
STANJE_NA_KRAJU_PRETH_KVARTALA    335954
STANJE_NA_KRAJU_KVARTALA               0
VALUTA                                 0
VRSTA_KLIJENTA                         0
PROIZVOD                               0
VRSTA_PROIZVODA                        0
VISINA_KAMATE                       4337
TIP_KAMATE                             0
STAROST                             1754
PRIJEVREMENI_RASKID                    0
dtype: int64

In [19]:
output2['NOVI_PDZ'] = np.where((output2['PRIJEVREMENI_RASKID']=='N') & (output2['PLANIRANI_DATUM_ZATVARANJA'].isnull()), output2['DATUM_ZATVARANJA'], output2['PLANIRANI_DATUM_ZATVARANJA'])

In [20]:
output2['PLANIRANI_DATUM_ZATVARANJA'] = output2['NOVI_PDZ']
del output2['NOVI_PDZ']

In [21]:
output2.shape

(3880690, 17)

In [45]:
df4 = output2[(output2['DATUM_OTVARANJA']<= output2['PLANIRANI_DATUM_ZATVARANJA']) & (output2['DATUM_OTVARANJA']<= output2['DATUM_ZATVARANJA'])]

In [23]:
df5 = df4.drop_duplicates( subset = ['KLIJENT_ID', 'OZNAKA_PARTIJE', 'DATUM_OTVARANJA'])

In [46]:
df5.groupby(['PRIJEVREMENI_RASKID']).size()

PRIJEVREMENI_RASKID
N    476900
Y      4504
dtype: int64

In [47]:
df4.sort_values(['KLIJENT_ID', 'OZNAKA_PARTIJE', 'DATUM_OTVARANJA', 'DATUM_IZVJESTAVANJA'], inplace = True)
grouped = df4.groupby(['KLIJENT_ID', 'OZNAKA_PARTIJE'])

df4['RASKID'] = grouped['PRIJEVREMENI_RASKID'].transform('last')

df4['PRODULJENJA'] = grouped['KLIJENT_ID'].transform('size') # svejedno po kojoj instanci gledam


In [48]:
df4.groupby(['RASKID']).size() / df4.shape[0]

RASKID
N    0.699508
Y    0.300492
dtype: float64

In [49]:
df = df4
# ovo ciniti nakon brisanja duplikata
grouped_kamate= df.groupby(['VRSTA_KLIJENTA', 'VRSTA_PROIZVODA'])
grouped_klijenti = df.groupby(['VRSTA_KLIJENTA'])
grouped =  df.groupby(['KLIJENT_ID', 'OZNAKA_PARTIJE'])

def impute_mean(series):
    return series.fillna(series.mean())

df['VISINA_KAMATE'] = grouped_kamate.transform(impute_mean) # mozda oni uzimaju prve instance samo
df['STAROST'] = grouped_klijenti.transform(impute_mean)

In [50]:
df.isnull().sum()

DATUM_IZVJESTAVANJA                    0
KLIJENT_ID                             0
OZNAKA_PARTIJE                         0
DATUM_OTVARANJA                        0
PLANIRANI_DATUM_ZATVARANJA             0
DATUM_ZATVARANJA                       0
UGOVORENI_IZNOS                        0
STANJE_NA_KRAJU_PRETH_KVARTALA    335949
STANJE_NA_KRAJU_KVARTALA               0
VALUTA                                 0
VRSTA_KLIJENTA                         0
PROIZVOD                               0
VRSTA_PROIZVODA                        0
VISINA_KAMATE                          0
TIP_KAMATE                             0
STAROST                                0
PRIJEVREMENI_RASKID                    0
RASKID                                 0
PRODULJENJA                            0
dtype: int64

In [51]:
# nakon sto imamo sve vrijednosti za PDZ, DZ i DO
df['GODINA_OTVARANJA'] = df['DATUM_OTVARANJA'].dt.year
df['PLANIRANA_DULJINA_KREDITA'] = (df['PLANIRANI_DATUM_ZATVARANJA']-df['DATUM_OTVARANJA']).dt.days
# df['DULJINA_KREDITA'] =  (df['DATUM_ZATVARANJA']-df['DATUM_OTVARANJA']).dt.days # toga nema u test-setu pa necemo koristiti

In [52]:
del df['DATUM_OTVARANJA']

In [53]:
df['PRIJEVREMENI_RASKID'] = df['RASKID']
del df['RASKID']

In [54]:
df.shape

(3880158, 19)

In [55]:
df.drop_duplicates(subset = ['KLIJENT_ID', 'OZNAKA_PARTIJE', 'DATUM_OTVARANJA'], inplace = True)

KeyError: Index(['DATUM_OTVARANJA'], dtype='object')

In [57]:
del_col_list = ['DATUM_IZVJESTAVANJA', 'PLANIRANI_DATUM_ZATVARANJA', 'DATUM_ZATVARANJA', 
                'STANJE_NA_KRAJU_PRETH_KVARTALA', 'STANJE_NA_KRAJU_KVARTALA']
df.drop(del_col_list, axis = 1, inplace = True)

In [59]:
df.groupby(['PRIJEVREMENI_RASKID']).size() / df.shape[0]

PRIJEVREMENI_RASKID
N    0.699508
Y    0.300492
dtype: float64

In [60]:
df.to_hdf('proba_train.h5', key = 'proba_train')

In [61]:
df.columns

Index(['KLIJENT_ID', 'OZNAKA_PARTIJE', 'UGOVORENI_IZNOS', 'VALUTA',
       'VRSTA_KLIJENTA', 'PROIZVOD', 'VRSTA_PROIZVODA', 'VISINA_KAMATE',
       'TIP_KAMATE', 'STAROST', 'PRIJEVREMENI_RASKID', 'PRODULJENJA',
       'GODINA_OTVARANJA', 'PLANIRANA_DULJINA_KREDITA'],
      dtype='object')